# Honey Bee Roundup

### Technical Goals:

Our technical goals are to analyze honeybee colony data from the USDA to determine if there is a statistically significant amount of loss among colonies over time, and to develop a machine learning model that accurately predicts honeybee colony loss based on significant environmental and human-driven features. We can use these predictions to make recommendations to stakeholders to minimize colony loss and improve outcomes.

### Business Goals:

Honeybees pollinate 15 billion worth of crops in the United States each year, including more than 130 types of fruits, nuts, and vegetables. Honeybees also produce honey, worth about 3.2 million in 2017 according to USDA-National Agricultural Statistics Service (NASS).

We want to provide stakeholders with an accurate model for predicting colony loss over time, which factors affect colony loss, and which areas are most conducive to colony production and preservation. We also want to provide a way for stakeholders to test outcomes based on actions they take (or not take) to mitigate colony loss. Our overarching business goal is to influence stakeholders to make responsible and proactive decisions to help honeybees thrive.

### Hypothesis

Our initial hypothesis is that honeybee colony loss has increased over time and will continue to increase year over year if no measures are taken to mitigate or reverse this outcome. Some initial questions we have are:

How much have honeybee colonies diminished over time? Is this loss compounded year over year?

What significant features drive honeybee colony loss?

What time of year is the biggest loss?

What state/area suffers heaviest loss and primary factors attributing to that?

Does summer or winter have the largest loss?

Does the beekeeper to colony ratio have an effect on colony loss?

### Executive Summary

We trained and evaluated four linear regression models using five significant features affecting colony loss. The OLS model and the LassoLars model both performed very well, beating the baseline RMSE for colony loss by {} percent. We selected the {} model because {}. The model's RMSE for the test set was {}. We used this model both with randomized data and chronological data, and the model performed similarly for both sets. We recommend that states use this model to predict colony losses for the upcoming year, and to lower their beekeeper to colony ratio for significant improvements. They can use this model to see how their losses will be mitigated if they increase the number of beekeepers and/or modify their beekeeping operations based on the season.

In [1]:
# standard and user-defined functions imports
import pandas as pd
import numpy as np
import wrangle
import explore
import regression_models as model
import os

# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# stats and modeling imports
from scipy import stats
from math import sqrt
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression,LassoLars,TweedieRegressor
from sklearn.feature_selection import SelectKBest, RFE, f_regression

# remove warnings
import warnings
warnings.filterwarnings("ignore")

## Acquisition

Data was originally found on Data.World, and further traced back to it's source to pull the most up-to-date data on Bees.

Of the original 3 Data sets:

- After assessement Census Data from the USDA was deemed not to have pertinent information to the current population of bees or health of hives, as well as the fact that it is reported in 5 year gaps, was removed from our initial data set.

- Survey Data by State from the USDA was also deemed as having less pertinent information on the trend in population of bees or health of their hives, and was left for further exploration as time permits due to a small amount of data on the environmental area of the colonies.

- The Bee Colony Loss Data from BeeInformed.org has the most relevant data regarding current number of colonies, number of bee keepers, and loss of bees and colonies, so was the data set we initially focused on. However, due to this privacy stipulation "For the protection of privacy, losses are reported as N/A if 10 or fewer beekeepers responded in that state. These beekeepers' losses are included in the national statistics." we decided to drop all records where the number of beekeepers were 10 or less, as most of the data was Null, and therefore not contributing towards our goal.

## Preparation

- We created separate functions for preparing the data- one for time series analysis and one for linear regression modeling. 
- All observations with 10 or less beekeepers were dropped because the data for those observations is protected under privacy laws. 
- All strings were stripped, lowercased, and spaces were replaced with underscores. 
- We engineered two columns: beekeeper_colony_ratio and colony_net_gain. 
- We also created dummy variables for the three categories in the season column: winter, summer, and annual.
- Multistate and non_continental data was dropped, and we isolated only observations with beekeepers exclusive to their respective states.

In [2]:
# acquire time series dataframe using function from wrangle.py and save to a variable
ts_df = wrangle.ts_bee_prep()

In [3]:
# confirm acquisition
ts_df

,state,year,season,beekeepers,total_loss,average_loss,starting_colonies,colonies_lost,ending_colonies,beekeepers_exclusive_to_state,colonies_exclusive_to_state,annual,summer,winter,colonies_net_gain,beekeeper_colony_ratio
104,alabama,2008-10-01,winter,16,41.916168,29.610216,4848,2240,3104,100.0,100.0,0,0,1,-1744,194.000000
107,arkansas,2008-10-01,winter,20,17.449588,12.786836,16955,3046,14410,100.0,100.0,0,0,1,-2545,720.500000
114,georgia,2008-10-01,winter,15,34.481800,29.086689,42876,18605,35351,100.0,100.0,0,0,1,-7525,2356.733333
118,iowa,2008-10-01,winter,12,40.248963,44.931741,723,291,432,100.0,100.0,0,0,1,-291,36.000000
123,maryland,2008-10-01,winter,14,7.560976,14.746298,4013,310,3790,100.0,100.0,0,0,1,-223,270.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,vermont,2022-10-01,winter,25,50.826045,43.504730,1019,523,506,100.0,100.0,0,0,1,-513,20.240000
7013,virginia,2022-10-01,winter,284,31.760436,37.625633,2463,875,1880,100.0,100.0,0,0,1,-583,6.619718
7014,washington,2022-10-01,winter,80,36.253776,51.579116,625,240,422,100.0,100.0,0,0,1,-203,5.275000
7015,west_virginia,2022-10-01,winter,27,54.261954,53.158014,427,261,220,100.0,100.0,0,0,1,-207,8.148148


In [4]:
# split data
train, validate, test = wrangle.ts_split(ts_df)

In [5]:
# verify split
train.shape, validate.shape, test.shape

((995, 16), (409, 16), (268, 16))

In [6]:
# set year column as datetime index
train = train.set_index('year').sort_index()
validate = validate.set_index('year').sort_index()
test = test.set_index('year').sort_index()

In [7]:
# confirm train data looks good
train

,state,season,beekeepers,total_loss,average_loss,starting_colonies,colonies_lost,ending_colonies,beekeepers_exclusive_to_state,colonies_exclusive_to_state,annual,summer,winter,colonies_net_gain,beekeeper_colony_ratio
year,,,,,,,,,,,,,,,
2008-10-01,alabama,winter,16,41.916168,29.610216,4848,2240,3104,100.0,100.0,0,0,1,-1744,194.000000
2008-10-01,wisconsin,winter,14,66.219008,51.694575,3472,2564,1308,100.0,100.0,0,0,1,-2164,93.428571
2008-10-01,west_virginia,winter,16,35.130581,27.295408,2174,834,1540,100.0,100.0,0,0,1,-634,96.250000
2008-10-01,utah,winter,28,24.781100,42.183693,17709,4500,13659,100.0,100.0,0,0,1,-4050,487.821429
2008-10-01,south_dakota,winter,15,48.920213,37.650885,80811,46574,48630,100.0,100.0,0,0,1,-32181,3242.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-01,maine,winter,80,45.498392,54.645797,613,283,339,100.0,100.0,0,0,1,-274,4.237500
2017-10-01,maryland,winter,149,43.715847,42.035286,1533,720,927,100.0,100.0,0,0,1,-606,6.221477
2017-10-01,massachusetts,winter,108,41.505792,50.766872,504,215,303,100.0,100.0,0,0,1,-201,2.805556


### Exploration

### Modeling

#### Scaling data

Scaling is the process by which we normalize the numeric range of the attributes of our data. 

In [8]:
# numeric columns to be scaled
columns = [col for col in train.drop(columns = ["state","season","colonies_lost"])]

In [9]:
#run scale data function to scale our numeric columns
scaled_train, scaled_validate, scaled_test = model.scale_data(train,validate,test,columns)

#### Feature selection

We are going to apply sklearn's SelectKbest and Recursive feature elimination to help us identify our top 5 features to fit into our model

##### Selectkbest

In [10]:
#Split the data to fit in the model
X= scaled_train[[col for col in scaled_train.columns if col.endswith("scaled")]]
y = scaled_train[["colonies_lost"]]

In [11]:
def select_kbest(X,y,k):
    """This function will input two array X, y and number of top features K and outputs the top k number of features """
    #create the model
    kbest = SelectKBest(f_regression, k=k)
    #fit the model
    kbest.fit(X,y)
    #output the top features 
    features = X.columns[kbest.get_support()]
    
    return features

In [12]:
#run the function
select_kbest(X,y,4)

Index(['starting_colonies_scaled', 'ending_colonies_scaled',
       'colonies_net_gain_scaled', 'beekeeper_colony_ratio_scaled'],
      dtype='object')

    These columns are our top features. Similarly, we can also apply recursive feature selection to help us identify our drivers

##### Recursive feature selection

In [13]:
def select_rfe(X,y,  n_features_to_select = 4):
    """This function will input two array X, y and number of top features desired and outputs those features """
    #create the model
    rfe=RFE(LinearRegression(), n_features_to_select = n_features_to_select) 
    #fit the model
    rfe.fit(X,y)
    #output top features
    features = X.columns[rfe.get_support()]
    
    return features

In [14]:
#run the function
select_rfe(X,y,  n_features_to_select = 4)

Index(['starting_colonies_scaled', 'ending_colonies_scaled',
       'colonies_net_gain_scaled', 'beekeeper_colony_ratio_scaled'],
      dtype='object')

    These featuers are identical to the features we obtain from Selectkbest. Hence we can use them to fit our regression models

In [17]:
#create list of  features to train the regression model with
features = ['starting_colonies_scaled', 'ending_colonies_scaled','colonies_net_gain_scaled', 'beekeeper_colony_ratio_scaled']

In [19]:
# X_train will be subset of our scaled train data with features only
X_train = scaled_train[features]
# set target
y_train = scaled_train[["colonies_lost"]]
# X_validate will be subset of our scaled validate data with features only
X_validate = scaled_validate[features]
# set target
y_validate = scaled_validate[["colonies_lost"]]
# X_test will be subset of our scaled test data with features only
X_test = scaled_test[features]
#set target
y_test = scaled_test[["colonies_lost"]]